In [9]:
import datetime
import networkx as nx
import pandas as pd
from pyvis import network as net

In [4]:
# read in json (converts to df)
path = "~/Desktop/SpotifyData/"

spotify_data = pd.read_json(path+"StreamingHistory0.json")

# set df as datetime
spotify_data['endTime'] = pd.to_datetime(spotify_data['endTime'])

# convert from milliseconds to seconds (for readibility)
spotify_data['seconds_played'] = spotify_data['msPlayed']/1000
print(spotify_data)

                 endTime               artistName            trackName  \
0    2020-04-20 20:11:00  Sammy Rae & The Friends         Denim Jacket   
1    2020-04-26 20:24:00                     Melt           Sour Candy   
2    2020-04-26 20:26:00              Beach Bunny               Sports   
3    2020-04-26 20:30:00              Demi Lovato            I Love Me   
4    2020-04-26 20:37:00  Sammy Rae & The Friends           Talk It Up   
...                  ...                      ...                  ...   
9406 2021-04-25 23:42:00              Monophonics    Last One Standing   
9407 2021-04-25 23:43:00        Kathryn Gallagher  I'll Get the Coffee   
9408 2021-04-25 23:46:00           Ben Thornewill            Songbirds   
9409 2021-04-25 23:48:00                Wes Reeve  Never-Ending Summer   
9410 2021-04-25 23:52:00               Emma Jayne              Morning   

      msPlayed  seconds_played  
0        83537          83.537  
1        50668          50.668  
2       1641

In [ ]:
# Get unique list of artists streamed + summarized count of seconds_played for a certain past time frame
# Use Spotipy to access artist genre data
# Summarize into a df - artist genres, seconds streamed in time frame
# Use df to create network graph of related artists potentially enlarged by stream time
# Prioritize top num of artists?

In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
cid = '0b07b283a2f943aa9ca99762b080e4bf'
secret = 'CLIENT_SECRET'
uri = 'http://localhost:8080/callback'

def get_user_top_artists(cid, secret, uri):
    #Reference: https://github.com/plamere/spotipy/blob/master/examples/my_top_artists.py
    scope = "user-library-read user-read-recently-played user-top-read"
    ranges = ['short_term', 'medium_term', 'long_term']

    # Access indiv spotify account (prompts sign in on localhost uri)
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=cid, client_secret=secret, redirect_uri=uri))

    # Get top artists
    top_artists = {}
    # Top artists docs: https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-users-top-artists-and-tracks
    for sp_range in ['short_term', 'medium_term', 'long_term']:
        results = sp.current_user_top_artists(time_range=sp_range, limit=25)
        top_artists[sp_range] = results['items']

    #     for i, item in enumerate(results['items']):
    #         print(i, item['name'], "\t", item['genres'], "\n")

    # Top artists lists ranked by popularity
    short_term_top = pd.DataFrame(top_artists['short_term'])
    med_term_top = pd.DataFrame(top_artists['medium_term'])
    long_term_top = pd.DataFrame(top_artists['long_term'])
    return short_term_top, med_term_top, long_term_top
    
short_term_top, med_term_top, long_term_top = get_user_top_artists(cid, secret, uri)


In [16]:
print(med_term_top)

                                        external_urls  \
0   {'spotify': 'https://open.spotify.com/artist/0...   
1   {'spotify': 'https://open.spotify.com/artist/1...   
2   {'spotify': 'https://open.spotify.com/artist/3...   
3   {'spotify': 'https://open.spotify.com/artist/2...   
4   {'spotify': 'https://open.spotify.com/artist/2...   
5   {'spotify': 'https://open.spotify.com/artist/2...   
6   {'spotify': 'https://open.spotify.com/artist/2...   
7   {'spotify': 'https://open.spotify.com/artist/5...   
8   {'spotify': 'https://open.spotify.com/artist/7...   
9   {'spotify': 'https://open.spotify.com/artist/2...   
10  {'spotify': 'https://open.spotify.com/artist/3...   
11  {'spotify': 'https://open.spotify.com/artist/3...   
12  {'spotify': 'https://open.spotify.com/artist/7...   
13  {'spotify': 'https://open.spotify.com/artist/3...   
14  {'spotify': 'https://open.spotify.com/artist/7...   
15  {'spotify': 'https://open.spotify.com/artist/6...   
16  {'spotify': 'https://open.s